In [2]:
==17==

In [ ]:
●파이썬은 필요 없어진 객체를 메모리에서 자동 삭제함.
●신경망의 메모리 관리는 큰 데이터를 다룬다
●파이썬의 메모리 관리방식: 1. 참조 수를 셈 2. 세대기준으로 쓸모없는 객체 회수(Garbage Collection)

In [103]:
import weakref
import numpy as np
from dezero.core_simple import Variable
import contextlib

In [100]:
a = np.array([1, 2, 3])
b = weakref.ref(a)

In [97]:
b

<weakref at 0x00000150EFCD9580; to 'numpy.ndarray' at 0x00000150F0C815F0>

In [15]:
b()

array([1, 2, 3])

In [17]:
a = None
b

<weakref at 0x00000150EFCD9620; to 'numpy.ndarray' at 0x00000150EFCC88D0>

In [12]:
class obj:
    pass

def f(x):
    print(x)
    
a =  obj() #변수에 대입: 참조카운트1
f(a) #함수에 전달: 함수 안에서 참조 카운트2
#함수 완료: 빠져 나오면 참조 카운트1
a =  None #대입 해제: 참조 카운트0

In [ ]:
a = obj()
b = obj()
c = obj()

a.b = b
b.c = c

a = b = c =None #a의 참조 카운트(b와c의 참조카운트는 1)

In [ ]:
#순환참조 (참조카운트로는 해결할 수 없는 문제)
a = obj()
b = obj()
c = obj()

a.b = b
a.c = c
c.a = a

a = b = c =None #메모리에서 삭제되지않음
##메모리 해제를 GC에 미루다 보면 메모리 사용량이 커지는 원인

In [80]:
class Variable:
    def __init__(self, data,name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0
        
    @property#shape메서드를 인스턴스 변수처럼 사용할 수 있게 함.
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype
    
    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def __len__(self):
        return len(self.data)

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self,retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()
        
        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  #outputs를 참조
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)
           
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y는 약한 참조
        @property
        def shape(self):
            return self.data.shape
        
Variable.__add__ = add
Variable.__mul__ = mul

In [59]:
class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs] #__call__메서드가 as_variable함수 이용 
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]
        
        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs]) #세대 설정
            for output in outputs:
                output.set_creator(self) #연결 설정
            self.inputs = inputs #인스턴스 변수 inputs는 역전파시 사용
            self.outputs = [weakref.ref(output) for output in outputs] #함수는 출력 변수를 약하게 참조
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

In [22]:
for i in range(10):
    x = Variable(np.random.randn(10000)) #거대한 데이터
    y = square(square(square(x))) #복잡한 계산 수행

In [ ]:
==18==
메모리사용 개선할 수 있는 구조
1. 역전파시 사용하는 메모리양을 줄임(불필요한 미분결과 보관하지않고 즉시 삭제)
2. 역전파가 필요없는 경우용 모드를 제공(불필요한 계산을 생략)

In [25]:
class Config:
    enable_backprop = True

In [37]:
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy

In [62]:
def add(x0, x1):
    x1 =  as_array(x1)
    return Add()(x0, x1)

In [40]:
x0=Variable(np.array(1.0))
x1=Variable(np.array(1.0))
t = add(x0, x1)
y = add(x0, t)
y.backward()

print(y.grad, t.grad)
print(x0.grad, x1.grad)

None None
2.0 1.0


In [27]:
#모드 전환 코드 수행
Config.enable_backprop = True #중간 계산 결과가 유지> 메모리 차지
x =  Variable(np.ones((100,100,100)))
y = square(square(x))
y.backward()

Config.enable_backprop = False #중간 계산 결과 바로 삭제
x = Variable(np.ones((100,100,100)))
y = square(square(x))

In [29]:
class Config:
    enable_backprop = True

In [30]:
@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)

In [31]:
with using_config('enable_backprop', False):
    x = Variable(np.array(2.0))
    y = square(x)

In [32]:
def no_grad():
    return using_config('enable_backprop', False)

In [33]:
with no_grad():
    x = Variable(np.array(2.0))
    y = square(x)
#>>기울기가 필요 없을떈 no_grad함수 호출

In [ ]:
==19==

In [ ]:
Variable클래스를 더욱 쉽게 사용하도록 개선
1. 변수 이름을 서로 구분
2. Variable 인스턴스를 ndarray인스턴스처럼 보이게 함(Variable인스턴스에서도 할 수 있도록 확장)
>>Variable클래스는 ndarray만을 취급
>>Variable이 데이터인 것처럼 보이게함
3. 파이썬의 len함수와 함께 사용
(_len_이라는 특수 메서드 구현시 Variable인스턴스에 대해서도 len함수 사용)
4.print함수를 사용해서 Variable안의 데이터 내용 출력
>>print함수가 출력해주는 문자열을 _repr_메서드를 재정의

In [44]:
x = np.array([[1, 2, 3], [4, 5, 6]])
x.shape #x.shape()대신 x.shape로 호출할 수 있다.

(2, 3)

In [46]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
print(len(x))

2


In [49]:
x=Variable(np.array([[1,2,3],[4,5,6]]))
print(x)

variable([[1 2 3]
          [4 5 6]])


In [ ]:
==20==

In [ ]:
+와 *연산자를 지원
곱셈을 수행하는 Mul클래스 구현

y=a*b처럼 코딩할 수 있도록 Variable인스턴스를 ndarray인스턴스처럼 사용하도록 구성

In [ ]:
곱셈의 순전파와 역전파
역전파는 최종출력인 L(손실)의 각 변수에 대한 미분을 전파

In [50]:
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0

In [51]:
def mul(x0, x1):
    return Mul()(x0, x1)

In [55]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = add(mul(a, b), c) #곱셈 사용! #y = a * b + c 이를 위해 연산자 오버로드 이용
y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


In [58]:
==21== 

In [ ]:
ndarray 인스턴스와 수치 데이터와도 함께 사용하도록 개선
a*np.array(2.0)처럼 ndarray인스턴스와 함께 사용할 수 없음
수치 데이터도 사용 불가
Variable인스턴스와 ndarray인스턴스 함께 사용 가능(int,float등도 함께 사용 가능)

In [71]:
def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

In [72]:
def add(x0,x1):
    x1 = as_array(x1)
    return Add()(x0,x1)

In [60]:
def as_variable(obj): #obj가 Variable인스턴스면 그대로 반환, 아니면 변환하여 반환
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

In [69]:
#y = 2.0 *x

In [75]:
Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul

In [77]:
# x = Variable(np.array(2.0))
# y = 3.0 * x + 1.0
# print(y)

In [ ]:
==22==

In [ ]:
연산자들을 추가
추가 순서
1. Function클래스를 상속하여 원하는 함수 클래스를 구현
2. 파이썬 함수로 사용할 수 있도록 함
3. Variable클래스의 연산자를 오버로드함

In [82]:
#음수(부호변환)
class Neg(Function):
    def forward(self, x):
        return -x

    def backward(self, gy):
        return -gy


def neg(x):
    return Neg()(x)

Variable.__neg__ = neg

In [83]:
x = Variable(np.array(2.0))
y = -x #부호를 바꾼다
print(y)

variable(-2.0)


In [84]:
#뺄셈
class Sub(Function): 
    def forward(self, x0, x1):
        y = x0 - x1
        return y

    def backward(self, gy):
        return gy, -gy

def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)

Variable.__sub__ = sub

In [85]:
def rsub(x0, x1):
    x1 = as_array(x1)
    return sub(x1, x0)

In [87]:
#나눗셈
class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1

In [88]:
def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)

In [89]:
def rdiv(x0, x1):
    x1 = as_array(x1)
    return div(x1, x0) #x0와 x1의 순서를 바꾼다.

In [90]:
Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv

In [91]:
#거듭제곱
class Pow(Function):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        y = x ** self.c
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c

        gx = c * x ** (c - 1) * gy
        return gx

In [92]:
def pow(x, c):
    return Pow(c)(x)
Variable.__pow__ = pow